In [163]:
from collections import defaultdict

In [164]:
import csv
import pandas as pd

In [165]:
df = pd.read_csv("rates_Step2_V38nano.csv")

In [166]:
l1_obj = []
thresholds = defaultdict(list)
with open("onl2off_th_update.txt") as f:
    for line in f:
        if "####" in line: continue
        if "L1_" in line:
            l1_obj.append(line)
            obj = line
        if "leg" in line:
            legs = line.split(',')
            _dict = {}
            _dict[legs[0].split(' Eta = ')[0]] = {'offline': legs[1].split(' = ')[1], 'online': legs[2].split(' = ')[1][:-1]}
            thresholds[obj[:-1]].append(_dict)
#         print(line) 

In [167]:
l1_thresholds = defaultdict(list)
rates = defaultdict(list)
for _obj in thresholds:
#     if _obj == 'L1_PFMHTT': continue # not in menu
    dfi= df[df.L1Path==_obj]
    if(len(dfi) == 0):
        print(f"... {_obj} missing ...")
        continue
    rates[_obj] = float(dfi.rate)
    offlines = []
    onlines = []
    for legs in thresholds[_obj]:
        for leg in legs:
            onlines.append(int(float(legs[leg]['online'])))
            offlines.append(int(float(legs[leg]['offline'])))
    l1_thresholds[_obj] = (onlines, offlines)

... L1_SingleTkMu missing ...
... L1_DoubleTkMu missing ...
... L1_TripleTkMu missing ...
... L1_TripleTkMu_NoSC missing ...
... L1_SingleTkEle missing ...
... L1_SingleTkEleIso missing ...
... L1_DoubleTkEle missing ...
... L1_SingleTkPhoIso missing ...
... L1_TkEleIso_EG missing ...
... L1_DoubleTkPhoIso missing ...
... L1_SinglePFTau missing ...
... L1_PFTau_PFTau missing ...
... L1_PFIsoTau_PFIsoTau missing ...
... L1_PFIsoTau_TkMu missing ...
... L1_TkEleIso_PFIsoTau missing ...
... L1_PFIsoTau_PFMet missing ...
... L1_SinglePfJet missing ...
... L1_DoublePFJet_dEtaMax missing ...
... L1_PFHTT missing ...
... L1_PFMHTT missing ...
... L1_PFMet missing ...
... L1_PFHTT_QuadJet missing ...
... L1_TkMu_TkEleIso missing ...
... L1_TkMu_TkEle missing ...
... L1_TkEle_TkMu missing ...
... L1_TkMu_DoubleTkEle missing ...
... L1_DoubleTkMu_TkEle missing ...
... L1_TkMu_PfHTT missing ...
... L1_TkMu_PfJet_dRMax_DoubleJet_dEtaMax missing ...
... L1_TkMu_PfJet_PfMet missing ...
... L1_Double

In [168]:
# for l1p in list(df.L1Path):
#     if l1p not in amap.keys():
#         print(l1p)

### Missing (or rather, no rate applied):
# "L1_TripleTkMu": ["Triple TkMuon", "?", "?"],
# "L1_TripleTkMu_5SQ_3SQ_0OQ_DoubleMu_5_3_SQ_OS_Mass_Max9_noSC": ["", "?", "?"],
# "L1_TripleTkMu_5_3p5_2p5_OS_Mass_5to17_noSC": ["Triple TkMuon 3.5,2.5 OS, no SC", "?", "?"],
# "L1_TripleTkMu_NoSC": ["Triple TkMuon no SC", "?", "?"],

In [169]:
l1_names_map = {
    "Single/Double/Triple Lepton (electron, muon) seeds":
    {"L1_SingleTkMu": ["Single tkMuon", r"$|\eta|<2.4$", "95"],
    "L1_DoubleTkMu": ["Double tkMuon", r"$|\eta|<2.4$, ${\Delta}z <1$", "95"],
#     "L1_DoubleTkMu9_SQ": ["\color{black}Double TkMuon 9 SQ", r"$|\eta|<2.4$", "?"],     
    "L1_SingleTkEle": ["Single tkElectron", r"$|\eta|<2.4$", "93"],
    "L1_SingleTkEleIso": ["Single tkIsoElectron", r"$|\eta|<2.4$", "93"],
    "L1_TkEleIso_EG": ["TkIsoElectron-StaEG", r"$|\eta|<2.4$", "93,99"],
    "L1_DoubleTkEle": ["Double tkElectron", r"$|\eta|<2.4$, ${\Delta}z <1$", "93"],
    "L1_SingleEGEle": ["Single StaEG", r"$|\eta|<2.4$", "99"],
    "L1_DoubleEGEle": ["Double StaEG", r"$|\eta|<2.4$", "99"],},
    
    "Photon seeds":
    {"L1_SingleTkPhoIso": ["Single TkIsoPhoton", r"$|\eta|<2.4$", "97"],
    "L1_DoubleTkPhoIso": ["Double TkIsoPhoton", r"$|\eta|<2.4$", "97"],},
    
    "Tau seeds":
    {"L1_SinglePFTau": ["Single CaloTau", r"$|\eta|<2.172$", "99"],
    "L1_PFTau_PFTau": ["Double CaloTau", r"$|\eta|<2.172$, ${\Delta}R >0.5$", "99"],
    "L1_PFIsoTau_PFIsoTau": ["Double PuppiTau", r"$|\eta|<2.172$, ${\Delta}R >0.5$, $\text{LooseNN} > 0$", "90"],},
    
    r"Hadronic seeds (jets,\HT)":
    {"L1_SinglePfJet": ["Single PuppiJet", r"$|\eta|<2.4$", "100"],
    "L1_DoublePFJet_dEtaMax": ["Double PuppiJet", r"$|\eta|<2.4$, ${\Delta}\eta <1.6$", "100"],
    "L1_PFHTT": ["Puppi\HT", r"jets: $|\eta|<2.4$, $\pt >30$", "100"],
    "L1_PFMHTT": [r"Puppi$\slashed{\ensuremath{H}}_{\mathrm{T}}$", r"jets: $|\eta|<2.4$, $\pt >30$", "100"],
    "L1_PFHTT_QuadJet": ["QuadPuppiJets-Puppi\HT(**)", r"jets: $|\eta|<2.4$, $\pt >25$", "100,100"],},

    r"\ETmiss seeds":
    {"L1_PFMet": [r"PuppiE_{T}^{miss}", "", "100"],},
    
    "Cross Lepton seeds":
    {"L1_TkMu_TkEleIso": ["TkMuon-TkIsoElectron", r"$|\eta|<2.4$, ${\Delta}z <1$", "95,93"],
    "L1_TkMu_TkEle": ["TkMuon-TkElectron", r"$|\eta|<2.4$, ${\Delta}z <1$", "95,93"],
    "L1_TkEle_TkMu": ["\color{black}TkElectron-TkMuon", r"$|\eta|<2.4$, ${\Delta}z <1$", "93,95"],
    "L1_TripleTkMu": ["Triple TkMuon", r"$|\eta|<2.4$, ${\Delta}z <1$, Qual $>$ 0", "95"],
    "L1_TkMu_DoubleTkEle": ["TkMuon-DoubleTkElectron", r"$|\eta|<2.4$, ${\Delta}z <1$", "95,93"],
    "L1_DoubleTkMu_TkEle": ["DoubleTkMuon-TkElectron", r"$|\eta|<2.4$, ${\Delta}z <1$, Qual $>$ 0", "95,93"],
    "L1_PFIsoTau_TkMu": ["PuppiTau-TkMuon", r" $|\eta|<2.172$, $|\eta|<2.1$, ${\Delta}z <1$, $\text{LooseNN} > 0$", "90,95"],
    "L1_TkEleIso_PFIsoTau": ["TkIsoElectron-PuppiTau", r"$|\eta|<2.172$, $|\eta|<2.1$, ${\Delta}z <1$, $\text{LooseNN} > 0$", "93,90"],},
    
    "Cross Hadronic-Lepton seeds":
    {"L1_TkMu_PfHTT": [r"TkMuon-Puppi\HT", r"$|\eta|<2.4$, ${\Delta}z <1$", "95,100"],
    "L1_TkMu_PfJet_dRMax_DoubleJet_dEtaMax": ["TkMuon-TriplePuppiJet", r"$|\eta|<2.4$, ${\Delta}R_{j1\mu}<0.4$,${\Delta}\eta_{j2j3}<1.6$, ${\Delta}z <1$", "95,100"],
    "L1_DoubleTkEle_PFHTT": ["DoubleTkElectron-Puppi\HT", r"$|\eta|<2.4$, ${\Delta}z <1$", "93,100"],
    "L1_DoubleTkMu_PfHTT": ["DoubleTkMuon-Puppi\HT", r"$|\eta|<2.4$, ${\Delta}z <1$, Qual $>$ 0", "93,100"],
    "L1_DoubleTkMu_PfJet_PfMet": ["DoubleTkMuon-PuppiJet-PuppiETmiss", r"$|\eta|<2.4$, ${\Delta}z <1$, Qual $>$ 0", "95,100,100"],
    "L1_TkEleIso_PFHTT": ["TkIsoElectron-Puppi\HT", r"$|\eta|<2.4$, ${\Delta}z <1$, $\text{LooseNN} > 0$", "93,100"],
    "L1_TkEle_PFJet_dRMin": ["TkElectron-PuppiJet (**)", r"$|\eta|<2.1$, $|\eta|<2.4$, ${\Delta}R>0.3$, ${\Delta}z <1$", "93,100"],
    "L1_PFIsoTau_PFMet": ["PuppiTau-PuppiE_{T}^{miss}", r"$|\eta|<2.172$, $\text{LooseNN} > 0$", "90,100"],
    "L1_TkMu_PfJet_PfMet": ["TkMuon-PuppiJet-PuppiETmiss", r"$|\eta|<2.4$, ${\Delta}z <1$, Qual $>$ 0", "95,100,100"],},
    
    "VBF seeds":
    {"L1_DoublePFJet_MassMin": ["Double PuppiJets (**)", r"$|\eta|<5$, $m_{jj}>620$", "100"]},
    
    "BPH seeds":
    {"L1_DoubleTkMu0er1p5_SQ_OS_dR_Max1p4": ["\color{black}Double TkMuon 0er1p5\_SQ\_OS\_dR\_Max1p4", r"$|\eta|<1.5$, ${\Delta}z <1$, ${\Delta}R <1.4$, $q_1\times q_2 <0$", "95"],
    "L1_DoubleTkMu4_SQ_OS_dR_Max1p2": ["\color{black}Double TkMuon 4\_SQ\_OS\_dR\_Max1p", r"$|\eta|<2.4$, ${\Delta}z <1$, ${\Delta}R <1.5$, $q_1\times q_2 <0$, Qual $>$ 0", "95"],
    "L1_DoubleTkMu4p5er2p0_SQ_OS_Mass7to18": ["\color{black}Double TkMuon 4p5er2p0\_SQ\_OS\_Mass7to18", r"$|\eta|<2.0$, ${\Delta}z <1$, $7 < m <18$, $q_1\times q_2 <0$, Qual $>$ 0", "95"],
    "L1_TripleTkMu_5SQ_3SQ_0OQ_DoubleMu_5_3_SQ_OS_Mass_Max9": ["Triple TkMuon 5SQ\_3SQ\_0OQ\_DoubleMu\_5\_3\_SQ\_OS\_Mass\_Max9", r"$|\eta|<2.4$, ${\Delta}z <1$, $p_{T,1} > 5$, $p_{T,2} > 3$, $p_{T,3} > 0$, $0< m <9$, $q_1\times q_2 <0$, Qual $>$ 0", "95"],
    "L1_TripleTkMu_5_3p5_2p5_OS_Mass_5to17": ["Triple TkMuon 5\_3p5\_2p5\_OS\_Mass\_5to17", r"$|\eta|<2.4$, ${\Delta}z <1$, $p_{T,1} > 5$, $p_{T,2} > 3.5$, $p_{T,3} > 2.5$, $5 <m <17$, $q_1\times q_2 <0$, Qual $>$ 0", "95"],
    "L1_PFHTT_QuadJet_BTagNNScore": ["QuadPuppiJets-Puppi\HT", r"$|\eta|<2.4$, Tot. b-tag score $>$ 2.2", "95"]
#     "L1_TripleTkMu_NoSC": ["Triple TkMuon no SC", r"$|\eta|<2.4$, ${\Delta}z <1$, $p_T > 3$, Qual $>$ 0", "95"]
    },
    
    "SC8 seeds":
    {"L1_SinglePfJet8": ["Single Ak8 PuppiJet", r"$|\eta|<2.4$", "100"],
    "L1_DoublePFJet8_dEtaMax": ["Double Ak8 PuppiJet", r"$|\eta|<2.4$, ${\Delta}\eta_{j1j2}<1.6$", "100"], 
    },
    
    "Displaced Muons":
    {
    "L1_SingleDispMu": ["Single Displaced Muon", r"$|\eta|<2.0$, Qual $\geq$ 15", "99"],
    "L1_DoubleDispMu": ["Single Displaced Muon", r"$|\eta|<2.4$", "99"],
    },
    
    "GMT":
    {
    "L1_W3pi_GTT": ["Track Triplet for W3Pi", "", "100"]
    }

    }

In [170]:
tot_rate = int(df[df.L1Path=='total menu'].rate)
# tot_rate_54 = tot_rate + 54
tot_rate_30pc = int(tot_rate + 0.3*tot_rate)
my_rates = []
# print('L1 Trigger seeds & Offline Threshold(s) at 90\% or 95\% [GeV] & Online Threshold(s) (Barrel) [kHz] & Rate [kHz] \\\\')
for seed in l1_names_map:
    print(f"\\hline \\multicolumn{{5}}{{|l|}}{{{seed}}} \\\\")
    for obj in l1_names_map[seed]:
        if obj not in l1_thresholds.keys(): continue
        print(f"\\hline {l1_names_map[seed][obj][0]}", end = " & ") #obj.replace('_', '\_'), end=" & ")
        print(*l1_thresholds[obj][1], end= " & ", sep = ",")
#         if "0.0" in l1_thresholds[obj][0]:
#             print("--", sep=",", end = " & ")
#         else:
#             print(*l1_thresholds[obj][0], sep=",", end = " & ")
        print(int(rates[obj]), end = " & ")
        my_rates.append(rates[obj])
        print(l1_names_map[seed][obj][1], end = " & ")
        print(l1_names_map[seed][obj][2], end = '\\\\\n')
    print("\\hline")

print("\\hline")
print(f"\multicolumn{{3}}{{|l}}{{Rate for above Trigger seeds}} &  \\multicolumn{{2}}{{r|}}{{{{\color{{black}}{tot_rate}}}}}\\\\")
print("\\hline")
# print(f"\multicolumn{{3}}{{|l}}{{Menu rate assuming 54 kHz from the excluded low-pT muon seeds }} &  \\multicolumn{{3}}{{r|}}{{{{\color{{red}}{tot_rate_54}}}}}\\\\")
# print("\\hline")
print(f"\multicolumn{{3}}{{|l}}{{\\bf Total \\Lone  Menu Rate (+30\\%)}} &  \\multicolumn{{2}}{{r|}}{{{{\color{{black}}{tot_rate_30pc}}}}}\\\\")
print("\\hline")

\hline \multicolumn{5}{|l|}{Single/Double/Triple Lepton (electron, muon) seeds} \\
\hline
\hline \multicolumn{5}{|l|}{Photon seeds} \\
\hline
\hline \multicolumn{5}{|l|}{Tau seeds} \\
\hline
\hline \multicolumn{5}{|l|}{Hadronic seeds (jets,\HT)} \\
\hline
\hline \multicolumn{5}{|l|}{\ETmiss seeds} \\
\hline
\hline \multicolumn{5}{|l|}{Cross Lepton seeds} \\
\hline
\hline \multicolumn{5}{|l|}{Cross Hadronic-Lepton seeds} \\
\hline
\hline \multicolumn{5}{|l|}{VBF seeds} \\
\hline
\hline \multicolumn{5}{|l|}{BPH seeds} \\
\hline QuadPuppiJets-Puppi\HT & 299,25,25,25,25 & 14 & $|\eta|<2.4$, Tot. b-tag score $>$ 2.2 & 95\\
\hline
\hline \multicolumn{5}{|l|}{SC8 seeds} \\
\hline Single Ak8 PuppiJet & 230 & 59 & $|\eta|<2.4$ & 100\\
\hline Double Ak8 PuppiJet & 200,200 & 13 & $|\eta|<2.4$, ${\Delta}\eta_{j1j2}<1.6$ & 100\\
\hline
\hline \multicolumn{5}{|l|}{Displaced Muons} \\
\hline Single Displaced Muon & 22 & 56 & $|\eta|<2.0$, Qual $\geq$ 15 & 99\\
\hline Single Displaced Muon & 15,7 & 14